# Description
### Goal of this script is to convert historical weather files stored in WY3 format into EPW format so it could be used for Building Energy Simulation

# Library Import

In [18]:
import os
import pandas as pd 
import glob
import numpy as np
from pathlib import Path

# Step 1 - Set File Paths

## Option 1 - Single Location Setup

In [16]:
#Path to historical weather data in wy3 format
CWEEDsPath=r'..\Weather Files - Before Conversion\CWEEDs\CAN_QC_AMQUI_7050145_CWEEDS2011_1998-2017.WY3'

#Path to typical weather data in epw format
CWECPath=r'..\Weather Files - Before Conversion\CWEC\CAN_QC_AMQUI_7050145_CWEC.EPW'

#Path to OutputFolder - if folder does not exist the subsequent scripts will create a folder
OutputFolder=r'..\Weather Files - After Conversion\CAN_QC_AMQUI_7050145'

## Option 2 - Batch Process Setup

In [19]:
#Path to folder with all Historical Weather Data in wy3 format
CWEEDsAllPath=Path(r'..\Weather Files - Before Conversion\CWEEDs')

#Path to folder with all Typical Year Weather Data in EPW format
CWECAllPath=Path(r'..\Weather Files - Before Conversion\CWEC')

OutputFolder=r'..\Weather Files - After Conversion\BatchOutput'

#CreatePath function creates a dataframe that matches the paths to historical and typical weather data based on weather station code used in the file name - please note the files have to be in the naming convention used by Environment and Climate Change Canada - 
WeatherFilePathList=CreatePath(CWEEDsAllPath,CWECAllPath)

# Step 2 - Export

## Option 1 - Single Location Export

In [17]:
#Converts weather data in wy3 format into epw format 
EPWDataset=WY3toEPW(CWEEDsPath)

#Splits histrorical weather data
EPWAssemble(EPWDataset, CWECPath, OutputFolder)

'Process Complete'

## Option 2 - Batch Processing 

In [20]:
#Runs Option 1 in a for loop - outputs files in separate folders based on location

for index, row in WeatherFilePathList.iterrows():
    CWEEDsPath=row['CWEEDsPath']
    CWECPath=row['CWECPath']
    Location=row['StationNumber']+ "_" + row['Location']
    OutputPath=os.path.join(OutputFolder,Location)

    
    EPWDataset=WY3toEPW(CWEEDsPath)
    EPWAssemble(EPWDataset, CWECPath, OutputPath)
    

# Functions

In [4]:
def CreatePath(CWEEDsAllPath1,CWECAllPath1):
    CWEEDsAllPath=CWEEDsAllPath1
    CWECAllPath=CWECAllPath1
    
    #List files in Folder
    CWEEDsAllPathList=pd.DataFrame(list(CWEEDsAllPath.glob('*')),columns=['Path'])
    CWECAllPathList=pd.DataFrame(list(CWECAllPath.glob('*')),columns=['Path'])

    #Converts Path to String and create new col for info extraction
    CWEEDsAllPathList['Pathstr']=CWEEDsAllPathList['Path'].astype('str') 
    CWECAllPathList['Pathstr']=CWECAllPathList['Path'].astype('str') 

    #Extract Station Number, Location and Year(for CWEEDs)
    CWEEDsAllPathList['Location']= CWEEDsAllPathList['Pathstr'].str.split(pat="_").str[2]
    CWEEDsAllPathList['StationNumber']= CWEEDsAllPathList['Pathstr'].str.split(pat="_").str[3]
    CWEEDsAllPathList['Years']= CWEEDsAllPathList['Pathstr'].str.split(pat="_").str[5].str.split(pat=".").str[0]

    CWECAllPathList['Location']= CWECAllPathList['Pathstr'].str.split(pat="_").str[2]
    CWECAllPathList['StationNumber']= CWECAllPathList['Pathstr'].str.split(pat="_").str[3]

    #Drop pathstr col
    CWEEDsAllPathList=CWEEDsAllPathList.drop('Pathstr',axis=1)
    CWECAllPathList=CWECAllPathList.drop('Pathstr',axis=1)

    #Merge CWEEDs and CWEC Path list
    WeatherFilePathList=pd.merge(CWEEDsAllPathList,CWECAllPathList, on='StationNumber')
    WeatherFilePathList=WeatherFilePathList.drop('Location_y',axis=1)

    WeatherFilePathList=WeatherFilePathList.rename(columns={"Path_x":"CWEEDsPath","Path_y":"CWECPath","Location_x":"Location"})

    WeatherFilePathList=WeatherFilePathList[["StationNumber","Location","Years","CWEEDsPath","CWECPath"]]

    WeatherFilePathList
    
    return WeatherFilePathList

In [5]:
def WY3toEPW(CWEEDsfilepath):

    FilePath=CWEEDsfilepath

    VarWidth = [7,1,4,2,2,2,4,4,2,4,2,4,2,4,1,4,1,4,1,4,1,2,1,4,1,4,1,4,1,8,1,5,1,4,1,4,1,3,1,4,1,2,1,2,1,1,1]
    VarName = ['Station','SourceCode','Year','Month','Day','Hour','ExtIr','GlobHorIr','GlobHorIrFlag','DirNormIr','DirNormIrFlag','DifHorIr','DifHorIrFlag','GloHorIllum',      'GloHorIllumFlag','DirNormIllum','DirNormIllumFlag','DifHorIllum','DifHorIllumFlag','ZenIll','ZenIllFlag','MinSun','MinSunFlag','CeilHeight','CeilHeightFlag','SkyCondition','SkyConditionFlag','Visibility','VisibilityFlag','PresentWeather','PresentWeatherFlag','StationPressure','StationPressureFlag','DryBulbTemp','DryBulbTempFlag','DewPointTemp','DewPointTempFlag','WindDir','WindDirFlag','WindSpeed','WindSpeedFlag','TotSkyCov','TotSkyCovFlag','OpSkyCov','OpSkyCovFlag','SnowCov','SnowCovFlag']
    df=[]
    df=pd.read_fwf(FilePath,widths=VarWidth,names=VarName,skiprows=1)

#df 
    epwVarName=['Year','Month','Day','Hour','Minute','Datasource','DryBulb','DewPoint','RelHum','AtmosPressure','ExtHorzRad','ExtDirRad','HorzIRSky','GloHorzRad','DirNormRad','DifHorzRad','GloHorzIllum','DirNormIllum','DifHorzIllum','ZenLum','WindDir','WindSpd','TotSkyCvr','OpaqSkyCvr','Visibility','CeilingHgt','PresWeathObs','PresWeathCodes','PrecipWtr','AerosolOptDepth','SnowDepth','DaysLastSnow','Albedo','Rain','RainQuantity']

    epw=[]
    epw=pd.DataFrame(columns=epwVarName)
    epw.Year=df.Year
    epw.Month=df.Month
    epw.Day=df.Day
    epw.Hour=df.Hour
    epw.Minute=0
    epw.Datasource='?9?9?9?9E0?9?9?9?9?9?9?9*9?9?9?9?9?9*9*_?9*9'
    epw.DryBulb=df.DryBulbTemp*0.1
    epw.DewPoint=df.DewPointTemp*0.1

    #Relative Humidity 
    ## Refer to http://www.bom.gov.au/climate/averages/climatology/relhum/calc-rh.pdf
    RH=[]

    RHVarName=['V','R','RelH']
    RH=pd.DataFrame(columns=RHVarName)
    RH.V=100*(np.exp(1.8096+((17.2694*epw.DewPoint)/(237.3+epw.DewPoint))))
    RH.R=np.exp(1.8096+((17.2694*epw.DryBulb)/(237.3+epw.DryBulb)))
    RH.RelH=RH.V/RH.R

    epw.RelHum=RH.RelH
    epw.AtmosPressure=10*df.StationPressure
    epw.ExtHorzRad=df.ExtIr/3.6
    epw.ExtDirRad='9999'
    epw.HorzIRSky='9999'
    epw.GloHorzRad=df.GlobHorIr/3.6
    epw.DirNormRad=df.DirNormIr/3.6
    epw.DifHorzRad=df.DifHorIr/3.6
    epw.GloHorzIllum=df.GloHorIllum*100
    epw.DirNormIllum=df.DirNormIllum*100
    epw.DifHorzIllum=df.DifHorIllum*100
    epw.ZenLum=df.ZenIll*100
    epw.WindDir=df.WindDir
    epw.WindSpd=df.WindSpeed/(10)
    epw.TotSkyCvr=df.TotSkyCov
    epw.OpaqSkyCvr=df.OpSkyCov
    epw.Visibility=df.Visibility/(10)
    epw.CeilingHgt=df.CeilHeight*10
    epw.PresWeathObs='9'
    epw.PresWeathCodes=df.PresentWeather
    epw.PrecipWtr='999'
    epw.AerosolOptDepth='999'
    epw.SnowDepth=df.SnowCov
    epw.DaysLastSnow='88'
    epw.Albedo='999'
    epw.Rain='999'
    epw.RainQuantity='99'

    epw.index=pd.to_datetime(epw[['Year', 'Month', 'Day', 'Hour', 'Minute']])

    return epw


In [6]:
def EPWAssemble(EPWDataset, CWECPath, OutputPath):

    Dataset=EPWDataset
    HeaderPath=CWECPath
    OutputFolder=OutputPath

    os.makedirs(OutputPath, exist_ok = True)


    with open(HeaderPath,"r") as file:
        Headerlines = file.readlines()[0:8]

    UniqueYears=Dataset['Year'].unique()
    i=0

    while i < len(UniqueYears):
        Year=UniqueYears[i]
        EPWDatasetYear=Dataset[Dataset['Year'] == UniqueYears[i]]
        WritePath = OutputFolder + f'\{Year}.csv'
        EPWDatasetYear.to_csv(WritePath, index=False, header=False)
        with open(WritePath, 'r') as file:
            Data = file.readlines()

        HeaderDataCombine=Headerlines+Data
        HeaderDataCombine=[row.replace('\n', '') for row in HeaderDataCombine]
        output_string = '\n'.join(HeaderDataCombine)

        with open(WritePath.replace('.csv', '.epw'), 'w') as outfile:
            outfile.write(output_string)
            os.remove(WritePath)
        i=i+1

    return 'Process Complete'